<a href="https://colab.research.google.com/github/dolmani38/Summary2/blob/main/EncodeDecoder_Test_0825.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 여기서부터 TEST

참조 : https://github.com/kiyoungkim1/LMkor


https://colab.research.google.com/drive/1WIk2bxglElfZewOHboPFNj8H44_VAyKE?usp=sharing#scrollTo=yoN2q0hZUbXN


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
#from torchtext.datasets import Multi30k
#from torchtext.data import Field, BucketIterator

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import spacy
import numpy as np

import random
import math
import time

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
import urllib.request
import nltk
nltk.download('punkt')

# 간단한 전처리
def clean_text(txt):
    txt = txt.replace('\n',' ')
    txt = txt.replace('\r',' ')    
    txt = txt.replace('=','')
    txt = txt.replace('\"','')   
    txt = txt.replace('\'','')
    #txt = txt.replace(',','')
    txt = txt.replace('..','')
    txt = txt.replace('...','')
    #txt = txt.replace('.','. ')
    txt = txt.replace('.','. ')
    txt = txt.replace('  ',' ')
    txt = txt.replace('  ',' ')    
    txt = txt.replace('  ',' ')   
    txt = txt.replace('  ',' ')           
    txt = txt.replace('  ',' ')
    txt = txt.replace('  ',' ')    
    txt = txt.replace('  ',' ')   
    txt = txt.replace('  ',' ')             
    return txt.strip()

def collect_training_dataset_for_s2s_generator(source_urls=[]):
    ko_sentences_dataset = []
    for url in source_urls:
        raw_text = urllib.request.urlopen(url).read().decode('utf-8')
        ko_sentences_dataset += nltk.sent_tokenize(clean_text(raw_text))

    sentences = []
    for txt in ko_sentences_dataset:
        txt = txt.strip()
        if len(txt) > 40 and txt.endswith('다.'):
            #ko_grammar_dataset.append([txt,1])
            txt = txt.replace('.','')
            sentences.append(txt)

    return sentences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
if True:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/summary/korean_news_corpus.csv')
df

,title,contents
0,문 대통령 변창흠 국토장관 사의표명 사실상 수용,정만호 국민소통수석이 12일 오후 청와대 춘추관 대브리핑룸에서 변창흠 국토부 장관 ...
1,계급장 수여하는 문 대통령,(아산뉴스1) 이광호 기자 문재인 대통령이 12일 오후 충남 아산시 경찰대학에서 열...
2,계급장 수여하는 문 대통령,(아산뉴스1) 이광호 기자 문재인 대통령이 12일 오후 충남 아산시 경찰대학에서 열...
3,수상자 메달 걸어주는 문 대통령,(아산뉴스1) 이광호 기자 문재인 대통령이 12일 오후 충남 아산시 경찰대학에서 열...
4,정몽구 서울아산병원에 50억 쾌척,인재 양성·소외 계층 지원 등 계획 “부친 질병·가난 악순환 끊기 원해 국내 최고 ...
...,...,...
140559,[건축과도시] 북한산을 캔버스 삼아. 미술관 또 하나의 작품이 되다,<은평구 진관동 사비나 미술관> 서울시 은평구 진관동에 자리잡은 사비나미술관. 삼각...
140560,조선후기 문인 김조순 별장 그린 옥호정도 첫 공개,국립중앙박물관 서화실 개편해 32점 새롭게 전시 옥호정도[국립중앙박물관 제공연합뉴스...
140561,안성 청룡사 대웅전에서 목재 곡자 발견,문화재청(청장 정재숙)의 국고보조와 기술지도로 안성시에서 시행하고 있는 안성 청룡사...
140562,156년전 ㄱ자 곡자 찾았다 안성 청룡사 기둥 밑에서,안성 청룡사 대웅전에서 발견된 곡자 【서울뉴시스】 이수지 기자 안성 청룡사 대웅전에...


In [6]:
import re
import sys
# 검사...
pattens = ["[34569][0-9]{3}[\;.\;-\; ][0-9]{4}[\;.\;-\; ][0-9]{4}[\;.\;-\; ][0-9]{4}",
           "[0-9]{2,3}[\:\s\;.\;,\;-;)][0-9]{3,4}[\:\s\;.\;,\;-][0-9]{4}",
           "[0-9]{1}[0-9]{1}[\W]?[0-1]{1}[0-9]{1}[\W]?[0-3]{1}[\W]?[0-9]{1}[\W]?[1-4]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}",
           "[0-9]{2}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{6}|[0-9]{3}[\:\s\;.\;,\;-]([0-9]{5,6}[\:\s\;.\;,\;-][0-9]{3}|[0-9]{6}[\:\s\;.\;,\;-][0-9]{5}|[0-9]{2,3}[\:\s\;.\;,\;-][0-9]{6}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{7}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{4,6}[\:\s\;.\;,\;-][0-9]|[0-9]{5}[\:\s\;.\;,\;-][0-9]{3}[\:\s\;.\;,\;-][0-9]{2}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{5}[\:\s\;.\;,\;-][0-9]{3}|[0-9]{4}[\:\s\;.\;,\;-][0-9]{4}[\:\s\;.\;,\;-][0-9]{3}|[0-9]{6}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{3}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{7})|[0-9]{4}[\:\s\;.\;,\;-]([0-9]{3}[\:\s\;.\;,\;-][0-9]{6}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{6}[\:\s\;.\;,\;-][0-9])|[0-9]{5}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{6}|[0-9]{6}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{5,6}"
           ]

filters = []
for p in pattens:
    filters.append(re.compile(p))

sentences = []

for raw_text in df['contents']:
    try:
        docs = nltk.sent_tokenize(clean_text(raw_text))
        for txt in docs:
            if txt.find('▶') > -1 or txt.find('@') > -1 or txt.find('ⓒ') > -1: 
                pass
            else:
                txt = txt.strip()
                if len(txt) > 40:
                    #ok = True
                    #for f in filters:
                    #    if len(f.findall(txt)) > 0:
                    #        ok = False
                    #        print('fitered:',txt)
                    #if ok:
                    if any(chr.isdigit() for chr in txt):
                        pass
                    else:
                        sentences.append(txt)
    except KeyboardInterrupt as ki:
        raise ki        
    except:
        print("Unexpected error:", sys.exc_info()[0])      



Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected error: <class 'AttributeError'>
Unexpected 

In [7]:
len(sentences)

1544643

In [8]:
import re
import sys
import io

#텍스트 정제(전처리)
def cleanText(readData):
    #텍스트에 포함되어 있는 특수 문자 제거
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', readData)
    return text

In [9]:
c_sentences = []
for sentence in sentences:
    s = cleanText(sentence)
    if len(s.split()) < 10:
        c_sentences.append(s)

In [10]:
len(c_sentences)

58552

In [11]:
c_sentences[1]

'김성수 과학기술정보통신부 과학기술혁신본부장이 간담회 참석자들과 기념촬영 하고 있다'

In [12]:
!pip3 install keybert
!pip3 install sentence-transformers
!pip3 install -q transformers

     |████████████████████████████████| 85 kB 4.0 MB/s 
     |████████████████████████████████| 209 kB 18.0 MB/s 
     |████████████████████████████████| 51 kB 9.5 MB/s 
     |████████████████████████████████| 2.6 MB 58.9 MB/s 
     |████████████████████████████████| 1.2 MB 57.6 MB/s 
     |████████████████████████████████| 43 kB 2.9 MB/s 
     |████████████████████████████████| 636 kB 67.4 MB/s 
     |████████████████████████████████| 3.3 MB 55.5 MB/s 
     |████████████████████████████████| 895 kB 46.2 MB/s 
  Created wheel for keybert: filename=keybert-0.4.0-py3-none-any.whl size=20359 sha256=440eec11fdaef1d51e5fdac591807754c5eab70a9601d10b1794abe59474a97c
  Stored in directory: /root/.cache/pip/wheels/fe/e1/7f/8ee72e851e92320fba5c68d9e6e28d227671876429bc80f420
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-py3-none-any.whl size=126710 sha256=d56a0d9c3d2daea3d8d2c54f3ec9d7815c959610af837e3e615a578f35cdb3ca
  Stored in directory: /root/.cache/pip/whee

In [13]:
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("xlm-r-large-en-ko-nli-ststb")

Downloading:   0%|          | 0.00/795 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.97k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/733 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [14]:
key_model = KeyBERT(model=sentence_model)

In [15]:
def extract_key(sentence):
    #print(s)
    top_n= int(len(sentence.split())/3)
    keys = key_model.extract_keywords(sentence,top_n=top_n)
    x= {}
    for k,p in keys:
        x[k] = sentence.lower().find(k)
        #print(x[k],k)
    keys = list(dict(sorted(x.items(), key=lambda item: item[1])).keys())
    return keys




In [52]:
i = 4
[' '.join(extract_key(c_sentences[i])),c_sentences[i]]

['이커머스기업인 이베이코리아가 매물로', '기자 네 이커머스기업인 이베이코리아가 매물로 나온 것은 지난해 하반기인데요']

In [17]:
pretraoned_kobert_model_name='kykim/bert-kor-base'
#tokenizer = KoBertTokenizer.from_pretrained(pretraoned_kobert_model_name)

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [35]:
from transformers import EncoderDecoderModel, BertTokenizerFast
import torch

tokenizer = BertTokenizerFast.from_pretrained(pretraoned_kobert_model_name)
model = EncoderDecoderModel.from_encoder_decoder_pretrained(pretraoned_kobert_model_name, pretraoned_kobert_model_name) # initialize Bert2Bert from pre-trained checkpoints

model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.eos_token_id = tokenizer.sep_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.encoder.vocab_size

model.config.max_length = 128
model.config.min_length = 64
model.config.no_repeat_ngram_size = 3
model.config.early_stopping = True
model.config.length_penalty = 2.0
model.config.num_beams = 4

Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertLMHeadModel: ['cls.seq_re

In [39]:
op = tokenizer("김성수 과학기술정보통신부 과학기술혁신본부장이 간담회 참석자들과 기념촬영 하고 있다", return_tensors="pt",padding="max_length", truncation=True, max_length=encoder_max_length)
print(op)
print("Tokens (str)      : {}".format([tokenizer.convert_ids_to_tokens(s) for s in op['input_ids'].tolist()[0]]))
print("Tokens (int)      : {}".format(op['input_ids'].tolist()[0]))
print("Tokens (attn_mask): {}\n".format(op['attention_mask'].tolist()[0]))

{'input_ids': tensor([[    2, 24499,  8069, 35077, 15954, 18931,  8210, 35077, 31246, 22116,
         14248, 31650,  8113, 17287, 30246, 16022, 19550, 14042, 13984,     3,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [66]:

encoder_max_length = 64
decoder_max_length = 128
batch_size = 8 #4 # 64, 128
dataset_iterator = []
batch_counter = 0

sentences2 = c_sentences[:1000]
for i in range(0,len(sentences2),batch_size):
#for i in range(0,1,batch_size):

    batch = {}
    batch['input_ids'] = []
    batch['attention_mask'] = []
    batch['decoder_input_ids'] = []
    batch['decoder_attention_mask'] = []
    batch['labels'] = []

    
    percent = ("{0:.2f}").format(100 * (i / float(len(sentences2))))
    print(f'\r {percent}% {i}/{str(len(sentences2))}', end="", flush=True)
    for j in range(i,i+batch_size):
        try:
            sentence = sentences2[j]+'.'
            keys = extract_key(sentence)
            #srct = tokenizer((' '.join(keys)).strip(), return_tensors="pt")['input_ids'].tolist()[0]
            #trgt = tokenizer(sentence, return_tensors="pt")['input_ids'].tolist()[0]
            # tokenize the inputs and labels
            inputs = tokenizer((' '.join(keys)).strip(), padding="max_length", truncation=True, max_length=encoder_max_length)
            outputs = tokenizer(sentence, padding="max_length", truncation=True, max_length=decoder_max_length)

            batch["input_ids"].append(inputs.input_ids)
            batch["attention_mask"].append(inputs.attention_mask)
            batch["decoder_input_ids"].append(outputs.input_ids)
            batch["decoder_attention_mask"].append(outputs.attention_mask)
            labels  = outputs.input_ids.copy()
    
            batch["labels"].append([-100 if token == tokenizer.pad_token_id else token for token in labels])

        except Exception as e:
            print('Error:',e)
            pass

    dataset_iterator.append(batch)



 99.20% 992/1000

In [67]:
torch.tensor(dataset_iterator[0]['decoder_input_ids'])

tensor([[    2, 15619, 29110,  ...,     0,     0,     0],
        [    2, 24499,  8069,  ...,     0,     0,     0],
        [    2, 39009, 21108,  ...,     0,     0,     0],
        ...,
        [    2, 15801,  5921,  ...,     0,     0,     0],
        [    2,  2046, 15257,  ...,     0,     0,     0],
        [    2, 40097, 21031,  ...,     0,     0,     0]])

# Train 준비

In [43]:
SRC_PAD_IDX = tokenizer.pad_token_id
TRG_PAD_IDX = tokenizer.pad_token_id

LEARNING_RATE = 0.0005

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [68]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):

        optimizer.zero_grad()
        
        ##print(src.shape)
        ##print(trg.shape)
        input_ids = torch.tensor(batch["input_ids"]).to(device)
        attention_mask = torch.tensor(batch["attention_mask"]).to(device)
        decoder_input_ids= torch.tensor(batch["decoder_input_ids"]).to(device)
        decoder_attention_mask= torch.tensor(batch["decoder_attention_mask"]).to(device)
        labels= torch.tensor(batch["labels"]).to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask,
                        decoder_input_ids=decoder_input_ids, 
                        decoder_attention_mask=decoder_attention_mask,
                        labels=labels)
        #print(outputs)
        loss, logits = outputs.loss, outputs.logits

        ##print(output.shape)
        #output = [batch size, trg len - 1, output dim]
        #trg = [batch size, trg len]
            
        #output_dim = output.shape[-1]
            
        #output = output.contiguous().view(-1, output_dim)
        #trg = trg[:,1:].contiguous().view(-1)
                
        #output = [batch size * trg len - 1, output dim]
        #trg = [batch size * trg len - 1]
            
        #loss = criterion(output, trg)
        
        loss.backward()
        
        ##print('loss',loss)
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        ##print('epoch_loss',epoch_loss)
    return epoch_loss / len(iterator)

In [25]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

# Train !!!!

In [70]:
bcnt = len(dataset_iterator)
print(bcnt)
train_iterator = dataset_iterator[0:bcnt-int(bcnt/10)]
valid_iterator = dataset_iterator[bcnt-int(bcnt/10):bcnt]
#train_iterator = dataset_iterator[0:bcnt-1]
#valid_iterator = dataset_iterator[bcnt-1:bcnt]
print('train',len(train_iterator))
print('valid',len(valid_iterator))

125
train 113
valid 12


In [56]:
def generate_summary(text):
    # cut off at BERT max length 512
    inputs = tokenizer(text, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    model.to(device)
    model.eval()
    outputs = model.generate(input_ids, attention_mask=attention_mask)

    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return output_str

In [57]:
generate_summary('이커머스기업인 이베이코리아가 매물로')

['##연연연타타타터터터타터타타다다다타타더더더다더다다더더 더 더 더더더원더더거더더분더더디더더대더더리더더데더더질더더드더더소더더간더더별더더도더더진더더개더더고더더서더더연더더성더더딱더더초더더만더더합더더본더더잘더더면더더구더더반더더라더더타더다원원원더원']

In [ ]:

N_EPOCHS = 100
CLIP = 1

best_valid_loss = float('inf')
model.to(device)

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    #valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    #if valid_loss < best_valid_loss:
    #    best_valid_loss = valid_loss
    #    torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    #print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
    print(generate_summary('이커머스기업인 이베이코리아가 매물로'))

Epoch: 01 | Time: 0m 50s
	Train Loss: 8.387 | Train PPL: 4388.702
['.....을..는..은..문화..가..의..이..에..와..를..해..했다..과..도시.. 공공..었다.. 있다.. 국제..도..']
Epoch: 02 | Time: 0m 52s
	Train Loss: 7.312 | Train PPL: 1497.994
['.....은..는..을.. 임상..를..가..문화..이..의.. 사례..에.. 제..와.. 아스..해..립니다..도..화..정..']
Epoch: 03 | Time: 0m 52s
	Train Loss: 7.186 | Train PPL: 1320.206
['...을..은.... 임상..가..보를..트라.. 아스..의..를..네.. 기다..카..와..는.. 둘러싼.. 사례..이.. 소중한..에..']
